# Exploratory Data Analysis

I have obtained code from three different kaggles.

- One Dataset contains matches that occurred in a team's home league. This dataset contains lots of statistics from each match and will be the one of the most useful datasets for this project. However, the league strength will not be used for anything in this as both teams that face eachother will be in the same league.

- The second data set is of the UEFA Champions League. The only useful data this set provides for this application is: both teams that played eachother, the date, and the final score. It will only contain matches that occured between teams with leagues based in Europe.

- The third data set is of the CONMEBOL Libertadores. This dataset also only contains columns for the date, teams, and score. This will only contain data for teams that play eachother in South America.

Finding data for the North American, Asian, and African Continental compitetions proved difficult. Without the luxury of time for manually scraping the data, It makes more sense to train the model on the data that is readily available, and hope that the model will be able to precisely predict the outcomes. 

There is also a slight issue with the home leagues dataset, as some of the leagues represented at the Club World Cup are not present in the dataset. For this case, I am hoping that metrics involving league and team strength will be able to achieve good predictions. 

## Clean and Combine the Data

The data needs to be cleaned since all of the tables contain different information. 

I am planning only on using data from the past three seasons to train the model. This is because I only have access to the current team and league strengths. In football, four years is still a very long time, but it is not expected for team's performance to change extremely drastically. So I believe that using current rankings can help to assess the difference between teams in the past four seasons. Since leagues across the globe start at different times, I will use Jan 1st, 2021 as the date to cutoff the data

As for the leagues table specifically, I will only need data for the leagues that have representatives at the 2025 FIFA Club World Cup.

In [184]:
import pandas as pd

# ids of only the leagues we want to use
league_id= ["E0","I1","D1","SP1","F1","BRA","P1","EGY","USA","AUS","ARG","TUN","MEX","JAP","SA","KOR","UAE","MOR","KSA","AUT"]

# Load league matches data
league_matches = pd.read_csv("../data/league_training_data_raw.csv",low_memory=False)

# Change the date to a datetime object
league_matches["MatchDate"] = pd.to_datetime(
    league_matches["MatchDate"],
    format="%Y-%m-%d")

# filter dataset by date and league id
league_matches = league_matches[
    (league_matches["MatchDate"] >= "2021-01-01") & 
    (league_matches["Division"].isin(league_id))
]

# Drop any rows that have important missing values
required_cols = ["MatchDate", "HomeTeam", "AwayTeam","Division", "FTHome", "FTAway", "FTResult"]

league_matches["HomeLeague"] = league_matches["Division"]
league_matches["AwayLeague"] = league_matches["Division"]

league_matches = league_matches.dropna(subset=required_cols)

# Remove any teams where they play less than 4 matches
# Stack HomeTeam and AwayTeam into one Series
team_appearances = pd.concat([league_matches["HomeTeam"], league_matches["AwayTeam"]])

# Count how many matches each team has played
team_match_counts = team_appearances.value_counts()

# Keep only teams with 4+ appearances
eligible_teams = team_match_counts[team_match_counts >= 4].index

league_matches = league_matches[
    (league_matches["HomeTeam"].isin(eligible_teams)) &
    (league_matches["AwayTeam"].isin(eligible_teams))
]

# select only columns that could be useful for training
league_matches = league_matches[
    [
        "MatchDate", "HomeTeam","HomeLeague", "AwayTeam", "AwayLeague",
        "FTHome", "FTAway", "FTResult", "HomeTarget", "AwayTarget",
        "HomeFouls","AwayFouls", "HomeCorners", "AwayCorners",
        "HomeYellow", "AwayYellow", "HomeRed", "AwayRed"
    ]
]

#Fix team names that are not consistent
team_name_map = {
    "MGladbach": "M'Gladbach",
    "M'gladbach": "M'Gladbach",
    "Nottm Forest": "Nott'm Forest",
    "Tigre": "Tigres",
    "U.A.N.L.- Tigres": "Tigres"
}

league_matches["HomeTeam"] = league_matches["HomeTeam"].replace(team_name_map)
league_matches["AwayTeam"] = league_matches["AwayTeam"].replace(team_name_map)

print(league_matches.head())
print(league_matches.shape)

home_teams = set(league_matches["HomeTeam"].unique())
away_teams = set(league_matches["AwayTeam"].unique())
all_teams = home_teams.union(away_teams)

print(f"Total unique teams: {len(all_teams)}")
print(sorted(all_teams))

        MatchDate       HomeTeam HomeLeague      AwayTeam AwayLeague  FTHome  \
177975 2021-01-01        Everton         E0      West Ham         E0     0.0   
177977 2021-01-01     Man United         E0   Aston Villa         E0     2.0   
177978 2021-01-02  Santos Laguna        MEX  Club America        MEX     1.0   
177984 2021-01-02      Tottenham         E0         Leeds         E0     3.0   
177988 2021-01-02     Villarreal        SP1       Levante        SP1     2.0   

        FTAway FTResult  HomeTarget  AwayTarget  HomeFouls  AwayFouls  \
177975     1.0        A         2.0         5.0        9.0        7.0   
177977     1.0        H         9.0         5.0       22.0       10.0   
177978     1.0        D         NaN         NaN        NaN        NaN   
177984     0.0        H         7.0         5.0       15.0       13.0   
177988     1.0        H         7.0         2.0       12.0       11.0   

        HomeCorners  AwayCorners  HomeYellow  AwayYellow  HomeRed  AwayRed  
177

### Cleaning Other Datasets to Combine

In [185]:
uefa_ucl_matches = pd.read_csv("../data/uefa_champions_league_training_data_raw.csv")

# Change the date to a datetime object
uefa_ucl_matches["DATE_TIME"] = pd.to_datetime(
    uefa_ucl_matches["DATE_TIME"],
    format="%d-%b-%y %I.%M.%S.%f %p"
)

uefa_ucl_matches["DATE"]= uefa_ucl_matches["DATE_TIME"].dt.date


#Fix team names that are not consistent and have special characters
team_name_map = {
    "AC Milan": "Milan",
    "Atlético Madrid":"Ath Madrid",
    "Bayern München":"Bayern Munich",
    "Beşiktaş":"Besiktas",
    "Borussia Dortmund":"Dortmund",
    "Chelsea FC":"Chelsea",
    "Dinamo Kiev":"Dynamo Kyiv",
    "FC Barcelona":"Barcelona",
    "FC Porto":"Porto",
    "Lille OSC":"Lille",
    "Liverpool FC":"Liverpool",
    "Malmö FF":"Malmo FF",
    "Manchester City":"Man City",
    "Manchester United":"Man United",
    "Paris Saint-Germain":"Paris SG",
    "RB Salzburg":"Salzburg",
    "SL Benfica":"Benfica",
    "Sevilla FC":"Sevilla",
    "Sporting CP":"Sp Lisbon",
    "VfL Wolfsburg":"Wolfsburg",
    "Villarreal CF":"Villarreal"
}

uefa_ucl_matches["HOME_TEAM"] = uefa_ucl_matches["HOME_TEAM"].replace(team_name_map)
uefa_ucl_matches["AWAY_TEAM"] = uefa_ucl_matches["AWAY_TEAM"].replace(team_name_map)

# Drop any rows that have important missing values
required_cols = ["DATE", "HOME_TEAM", "AWAY_TEAM", "HOME_TEAM_SCORE", "AWAY_TEAM_SCORE"]

uefa_ucl_matches = uefa_ucl_matches.dropna(subset=required_cols)

# select only columns that could be useful for training
uefa_ucl_matches = uefa_ucl_matches[required_cols]

# filter dataset by date
uefa_ucl_matches = uefa_ucl_matches[
    (uefa_ucl_matches["DATE"] >= pd.to_datetime("2021-01-01").date())
]

team_appearances = pd.concat([uefa_ucl_matches["HOME_TEAM"], uefa_ucl_matches["AWAY_TEAM"]])

# Count how many matches each team has played
team_match_counts = team_appearances.value_counts()

# Keep only teams with 4+ appearances
eligible_teams = team_match_counts[team_match_counts >= 4].index

uefa_ucl_matches = uefa_ucl_matches[
    (uefa_ucl_matches["HOME_TEAM"].isin(eligible_teams)) &
    (uefa_ucl_matches["AWAY_TEAM"].isin(eligible_teams))
]

print(uefa_ucl_matches.head())
print(uefa_ucl_matches.shape)

home_teams = set(uefa_ucl_matches["HOME_TEAM"].unique())
away_teams = set(uefa_ucl_matches["AWAY_TEAM"].unique())
all_teams = home_teams.union(away_teams)

print(f"Total unique teams: {len(all_teams)}")
print(sorted(all_teams))

         DATE       HOME_TEAM       AWAY_TEAM  HOME_TEAM_SCORE  \
0  2021-09-15        Man City      RB Leipzig                6   
1  2021-09-15  Club Brugge KV        Paris SG                1   
2  2021-09-28        Paris SG        Man City                2   
3  2021-09-28      RB Leipzig  Club Brugge KV                1   
4  2021-10-19  Club Brugge KV        Man City                1   

   AWAY_TEAM_SCORE  
0                3  
1                1  
2                0  
3                2  
4                5  
(150, 5)
Total unique teams: 32
['AFC Ajax', 'Atalanta', 'Ath Madrid', 'BSC Young Boys', 'Barcelona', 'Bayern Munich', 'Benfica', 'Besiktas', 'Chelsea', 'Club Brugge KV', 'Dortmund', 'Dynamo Kyiv', 'FC Sheriff', 'Inter', 'Juventus', 'Lille', 'Liverpool', 'Malmo FF', 'Man City', 'Man United', 'Milan', 'Paris SG', 'Porto', 'RB Leipzig', 'Real Madrid', 'Salzburg', 'Sevilla', 'Shakhtar Donetsk', 'Sp Lisbon', 'Villarreal', 'Wolfsburg', 'Zenit St. Petersburg']


In [186]:
comnebol_matches = pd.read_csv("../data/conmebol_libertadores_training_raw.csv")

# Change the date to a datetime object
comnebol_matches["Date"] = pd.to_datetime(
    comnebol_matches["Date"],
    format="%d/%m/%Y"
)

# filter dataset by date
comnebol_matches = comnebol_matches[
    (comnebol_matches["Date"] >= pd.to_datetime("2021-01-01"))
]

team_appearances = pd.concat([comnebol_matches["Home Club"], comnebol_matches["Away Club"]])

# Count how many matches each team has played
team_match_counts = team_appearances.value_counts()

# Keep only teams with 4+ appearances
eligible_teams = team_match_counts[team_match_counts >= 4].index

comnebol_matches = comnebol_matches[
    (comnebol_matches["Home Club"].isin(eligible_teams)) &
    (comnebol_matches["Away Club"].isin(eligible_teams))
]

# Drop any rows that have important missing values
required_cols = ["Date", "Home Club", "Away Club", "Home Score", "AwayScore"]

comnebol_matches = comnebol_matches.dropna(subset=required_cols)

# select only columns that could be useful for training
comnebol_matches = comnebol_matches[required_cols]

#Fix team names that are not consistent and have special characters
team_name_map = {
    "América Mineiro":"America MG",
    "América de Cali":"America de Cali",
    "Argentinos Juniors":"Argentinos Jrs",
    "Atlético Mineiro":"Atletico-MG",
    "Atlético Nacional": "Atletico Nacional",
    "Atlético PR":"Athletico-PR",
    "Bolívar":"Bolivar",
    "Botafogo FR":"Botafogo RJ",
    "Cerro Porteño":"Cerro Porteno",
    "Deportivo Táchira":"Deportivo Tachira",
    "Estudiantes de la Plata":"Estudiantes L.P.",
    "Flamengo":"Flamengo RJ",
    "Fluminense FC":"Fluminense",
    "Fortaleza Esporte Clube":"Fortaleza",
    "Grêmio":"Gremio",
    "Huracán":"Huracan",
    "Independiente Medellín":"Independiente Medellin",
    "Liverpool":"Liverpool Montevideo",
    "Nacional":"Nacional Uruguay",
    "Peñarol":"Penarol",
    "Red Bull Bragantino SP":"Bragantino",
    "San Lorenzo de Almagro":"San Lorenzo",
    "Santos FC":"Santos",
    "São Paulo":"Sao Paulo",
    "Talleres de Cordoba":"Talleres Cordoba",
    "Universidad Católica":"Universidad Catolica",
    "Unión La Calera":"Union La Calera",
    "Vélez Sarsfield":"Velez Sarsfield",
    "Ñublense":"Nublense"
}

comnebol_matches["Home Club"] = comnebol_matches["Home Club"].replace(team_name_map)
comnebol_matches["Away Club"] = comnebol_matches["Away Club"].replace(team_name_map)

print(comnebol_matches.head())
print(comnebol_matches.shape)

home_teams = set(comnebol_matches["Home Club"].unique())
away_teams = set(comnebol_matches["Away Club"].unique())
all_teams = home_teams.union(away_teams)

print(f"Total unique teams: {len(all_teams)}")
print(sorted(all_teams))


        Date      Home Club      Away Club  Home Score  AwayScore
0 2023-11-04     Fluminense   Boca Juniors           2          1
1 2023-10-06      Palmeiras   Boca Juniors           1          1
2 2023-10-05  Internacional     Fluminense           1          2
3 2023-09-29   Boca Juniors      Palmeiras           0          0
4 2023-09-28     Fluminense  Internacional           2          2
(609, 5)
Total unique teams: 69
['Alianza Lima', 'Always Ready', 'America MG', 'America de Cali', 'Argentinos Jrs', 'Athletico-PR', 'Atletico Nacional', 'Atletico-MG', 'Aucas', 'Barcelona SC', 'Boca Juniors', 'Bolivar', 'Botafogo RJ', 'Bragantino', 'CD Magallanes', 'CS Deportivo Pereira', 'Caracas', 'Cerro Porteno', 'Cobresal', 'Colo Colo', 'Colón Santa Fe', 'Corinthians', 'Defensa y Justicia', 'Deportivo Cali', 'Deportivo La Guaira FC', 'Deportivo Tachira', 'Emelec', 'Estudiantes L.P.', 'FBC Melgar', 'Flamengo RJ', 'Fluminense', 'Fortaleza', 'Gremio', 'Huachipato', 'Huracan', 'Independiente', 'In

Now that I have added all the team and league strengths by hand, now we can begin to combine all of the datasets into a table that can be used to train the model.

### Combining Datasets

To combine, we need to make sure that all of them have the same feature names and the same number of features. We should fill any missing data in with `Na`s

In [187]:
print(league_matches.columns.tolist())
print(uefa_ucl_matches.columns.tolist())
print(comnebol_matches.columns.tolist())

['MatchDate', 'HomeTeam', 'HomeLeague', 'AwayTeam', 'AwayLeague', 'FTHome', 'FTAway', 'FTResult', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed']
['DATE', 'HOME_TEAM', 'AWAY_TEAM', 'HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE']
['Date', 'Home Club', 'Away Club', 'Home Score', 'AwayScore']


Since we have different column names and different amounts of features, we should make them have the same name and also the same amount of variables, filling empty ones with `Na` when required.

We should also add in the home and away team strengths and league strengths for the tables.

In [188]:

# rename columns in the copa libertadores table
comnebol_matches.rename(columns={'Date':"DATE",'Home Club':"HOME_TEAM",'Away Club':"AWAY_TEAM",'Home Score':"HOME_TEAM_SCORE",'AwayScore':"AWAY_TEAM_SCORE"},inplace=True)

continental_matches = pd.concat([uefa_ucl_matches,comnebol_matches],axis=0)
continental_matches["DATE"] = pd.to_datetime(continental_matches["DATE"])
continental_matches=continental_matches.sort_values(by='DATE')

continental_matches.shape
continental_matches.head(10)

,DATE,HOME_TEAM,AWAY_TEAM,HOME_TEAM_SCORE,AWAY_TEAM_SCORE
758,2021-01-06,Boca Juniors,Santos,0,0
759,2021-01-06,River Plate,Palmeiras,0,3
756,2021-01-13,Santos,Boca Juniors,3,0
757,2021-01-13,Palmeiras,River Plate,0,2
755,2021-01-30,Palmeiras,Santos,1,0
222,2021-02-16,Barcelona,Paris SG,1,4
221,2021-02-16,RB Leipzig,Liverpool,0,2
224,2021-02-17,Sevilla,Dortmund,2,3
223,2021-02-17,Porto,Juventus,2,1
226,2021-02-23,Ath Madrid,Chelsea,0,1


Now, we have combined the data for the South American and the European Continential competitions. This will reduce the amount of joins we need to do by half. 

Now we need to add in the team strengths. This join will be on the Team names. The `team_strength` table will also bring information about which league the team is in and the strength of the team. 

In [189]:
team_strengths = pd.read_csv("../data/team_strength.csv")

home_strengths = team_strengths.copy()
away_strengths = team_strengths.copy()

home_strengths.rename(columns={"country_abbrev":"HOME_LEAGUE_ID","team_name":"HOME_TEAM","strength":"HOME_STRENGTH"},inplace=True)
away_strengths.rename(columns={"country_abbrev":"AWAY_LEAGUE_ID","team_name":"AWAY_TEAM","strength":"AWAY_STRENGTH"},inplace=True)

# Merge to get team strengths
continental_matches = continental_matches.merge(home_strengths, on="HOME_TEAM", how="left")
continental_matches = continental_matches.merge(away_strengths, on="AWAY_TEAM", how="left")


continental_matches.head()

,DATE,HOME_TEAM,AWAY_TEAM,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,HOME_LEAGUE_ID,HOME_STRENGTH,AWAY_LEAGUE_ID,AWAY_STRENGTH
0,2021-01-06,Boca Juniors,Santos,0,0,ARG,82.0,BRA,75.7
1,2021-01-06,River Plate,Palmeiras,0,3,ARG,81.0,BRA,83.4
2,2021-01-13,Santos,Boca Juniors,3,0,BRA,75.7,ARG,82.0
3,2021-01-13,Palmeiras,River Plate,0,2,BRA,83.4,ARG,81.0
4,2021-01-30,Palmeiras,Santos,1,0,BRA,83.4,BRA,75.7


Now we have the `continental_matches` table which now has team strengths and league Ids for both home and away teams. Now we can join on the league ids to add in the league strength.

In [190]:
league_strengths = pd.read_csv("../data/league_strength.csv")

home_league_strengths = league_strengths.copy()
away_league_strengths = league_strengths.copy()

home_league_strengths.rename(columns={"league_abbrev":"HOME_LEAGUE_ID","league_name":"HOME_TEAM_LEAGUE","strength":"HOME_LEAGUE_STRENGTH"},inplace=True)
away_league_strengths.rename(columns={"league_abbrev":"AWAY_LEAGUE_ID","league_name":"AWAY_TEAM_LEAGUE","strength":"AWAY_LEAGUE_STRENGTH"},inplace=True)

# Merge to get league strengths
continental_matches = continental_matches.merge(home_league_strengths, on="HOME_LEAGUE_ID", how="left")
continental_matches = continental_matches.merge(away_league_strengths, on="AWAY_LEAGUE_ID", how="left")

continental_matches.head()


,DATE,HOME_TEAM,AWAY_TEAM,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,HOME_LEAGUE_ID,HOME_STRENGTH,AWAY_LEAGUE_ID,AWAY_STRENGTH,HOME_TEAM_LEAGUE,HOME_LEAGUE_STRENGTH,AWAY_TEAM_LEAGUE,AWAY_LEAGUE_STRENGTH
0,2021-01-06,Boca Juniors,Santos,0,0,ARG,82.0,BRA,75.7,Liga Professional,76.8,Serie A,80.1
1,2021-01-06,River Plate,Palmeiras,0,3,ARG,81.0,BRA,83.4,Liga Professional,76.8,Serie A,80.1
2,2021-01-13,Santos,Boca Juniors,3,0,BRA,75.7,ARG,82.0,Serie A,80.1,Liga Professional,76.8
3,2021-01-13,Palmeiras,River Plate,0,2,BRA,83.4,ARG,81.0,Serie A,80.1,Liga Professional,76.8
4,2021-01-30,Palmeiras,Santos,1,0,BRA,83.4,BRA,75.7,Serie A,80.1,Serie A,80.1


Now we have all the joins for this table completed. Now we need to work on adding empty features so we can combine with the `league_matches` table. We should also create a results feature, which can be done by comparing the `HOME_TEAM_SCORE` and the `AWAY_TEAM_SCORE`.

In [191]:
import numpy as np

conditions = [
    continental_matches["AWAY_TEAM_SCORE"] > continental_matches["HOME_TEAM_SCORE"],
    continental_matches["AWAY_TEAM_SCORE"] < continental_matches["HOME_TEAM_SCORE"],
    continental_matches["AWAY_TEAM_SCORE"] == continental_matches["HOME_TEAM_SCORE"]
]

choices = ["A", "H", "D"]

continental_matches["RESULT"] = np.select(conditions, choices, default=pd.NA)

new_columns = ['HOME_TARGET', 'AWAY_TARGET', 'HOME_FOULS', 'AWAY_FOULS', 'HOME_CORNERS', 'AWAY_CORNERS', 'HOME_YELLOW', 'AWAY_YELLOW', 'HOME_RED', 'AWAY_RED']
for col in new_columns:
    continental_matches[col] = np.nan

#Order columns in a way that is easily readable
new_order = ["DATE","HOME_TEAM","AWAY_TEAM","RESULT","HOME_TEAM_SCORE","AWAY_TEAM_SCORE","HOME_TEAM_LEAGUE","HOME_LEAGUE_ID",
    "HOME_STRENGTH","HOME_LEAGUE_STRENGTH","AWAY_TEAM_LEAGUE","AWAY_LEAGUE_ID","AWAY_STRENGTH","AWAY_LEAGUE_STRENGTH","HOME_TARGET",
    "AWAY_TARGET","HOME_FOULS","AWAY_FOULS","HOME_CORNERS","AWAY_CORNERS","HOME_YELLOW","AWAY_YELLOW","HOME_RED",
    "AWAY_RED"]

continental_matches = continental_matches.reindex(columns=new_order)

continental_matches.head()

,DATE,HOME_TEAM,AWAY_TEAM,RESULT,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,HOME_TEAM_LEAGUE,HOME_LEAGUE_ID,HOME_STRENGTH,HOME_LEAGUE_STRENGTH,...,HOME_TARGET,AWAY_TARGET,HOME_FOULS,AWAY_FOULS,HOME_CORNERS,AWAY_CORNERS,HOME_YELLOW,AWAY_YELLOW,HOME_RED,AWAY_RED
0,2021-01-06,Boca Juniors,Santos,D,0,0,Liga Professional,ARG,82.0,76.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-06,River Plate,Palmeiras,A,0,3,Liga Professional,ARG,81.0,76.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-13,Santos,Boca Juniors,H,3,0,Serie A,BRA,75.7,80.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-13,Palmeiras,River Plate,A,0,2,Serie A,BRA,83.4,80.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-30,Palmeiras,Santos,H,1,0,Serie A,BRA,83.4,80.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Format the League Matches Table

In [192]:
print(league_matches.columns.tolist())
print(continental_matches.columns.tolist())

['MatchDate', 'HomeTeam', 'HomeLeague', 'AwayTeam', 'AwayLeague', 'FTHome', 'FTAway', 'FTResult', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed']
['DATE', 'HOME_TEAM', 'AWAY_TEAM', 'RESULT', 'HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE', 'HOME_TEAM_LEAGUE', 'HOME_LEAGUE_ID', 'HOME_STRENGTH', 'HOME_LEAGUE_STRENGTH', 'AWAY_TEAM_LEAGUE', 'AWAY_LEAGUE_ID', 'AWAY_STRENGTH', 'AWAY_LEAGUE_STRENGTH', 'HOME_TARGET', 'AWAY_TARGET', 'HOME_FOULS', 'AWAY_FOULS', 'HOME_CORNERS', 'AWAY_CORNERS', 'HOME_YELLOW', 'AWAY_YELLOW', 'HOME_RED', 'AWAY_RED']


In [193]:
league_matches.rename(columns={"MatchDate":"DATE","HomeTeam":"HOME_TEAM","AwayTeam":"AWAY_TEAM",
                               "HomeLeague":"HOME_LEAGUE_ID","AwayLeague":"AWAY_LEAGUE_ID", "FTHome":"HOME_TEAM_SCORE",
                               "FTAway":"AWAY_TEAM_SCORE","FTResult":"RESULT", "HomeTarget":"HOME_TARGET",
                               "AwayTarget":"AWAY_TARGET","HomeFouls":"HOME_FOULS","AwayFouls":"AWAY_FOULS",
                               "HomeCorners":"HOME_CORNERS","AwayCorners":"AWAY_CORNERS","HomeYellow":"HOME_YELLOW",
                               "AwayYellow":"AWAY_YELLOW","HomeRed":"HOME_RED","AwayRed":"AWAY_RED"}, inplace=True)

#Merge to get league strengths
league_matches = league_matches.merge(home_league_strengths, on="HOME_LEAGUE_ID", how="left")
league_matches = league_matches.merge(away_league_strengths, on="AWAY_LEAGUE_ID", how="left")

league_matches = league_matches.merge(home_strengths, on=["HOME_TEAM","HOME_LEAGUE_ID"], how="left")
league_matches = league_matches.merge(away_strengths, on=["AWAY_TEAM","AWAY_LEAGUE_ID"], how="left")

league_matches = league_matches.reindex(columns=new_order)

print(league_matches.shape)
league_matches.head()


(18031, 24)


,DATE,HOME_TEAM,AWAY_TEAM,RESULT,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,HOME_TEAM_LEAGUE,HOME_LEAGUE_ID,HOME_STRENGTH,HOME_LEAGUE_STRENGTH,...,HOME_TARGET,AWAY_TARGET,HOME_FOULS,AWAY_FOULS,HOME_CORNERS,AWAY_CORNERS,HOME_YELLOW,AWAY_YELLOW,HOME_RED,AWAY_RED
0,2021-01-01,Everton,West Ham,A,0.0,1.0,Premier League,E0,86.3,85.9,...,2.0,5.0,9.0,7.0,5.0,4.0,2.0,0.0,0.0,0.0
1,2021-01-01,Man United,Aston Villa,H,2.0,1.0,Premier League,E0,88.6,85.9,...,9.0,5.0,22.0,10.0,5.0,10.0,4.0,0.0,0.0,0.0
2,2021-01-02,Santos Laguna,Club America,D,1.0,1.0,Liga MX,MEX,73.0,75.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-02,Tottenham,Leeds,H,3.0,0.0,Premier League,E0,87.8,85.9,...,7.0,5.0,15.0,13.0,3.0,5.0,2.0,1.0,1.0,0.0
4,2021-01-02,Villarreal,Levante,H,2.0,1.0,LaLiga,SP1,87.1,83.5,...,7.0,2.0,12.0,11.0,3.0,5.0,1.0,1.0,0.0,0.0


In [194]:
# Combine the two datasets and save it

match_data = pd.concat([continental_matches,league_matches])
match_data=match_data.sort_values(by='DATE')

# remove entries without a team rating
match_data = match_data.dropna(subset=["HOME_STRENGTH", "AWAY_STRENGTH"])

match_data.to_csv("../data/final_match_data.csv",index=False)

print(match_data.shape)
match_data.head()

(16818, 24)


,DATE,HOME_TEAM,AWAY_TEAM,RESULT,HOME_TEAM_SCORE,AWAY_TEAM_SCORE,HOME_TEAM_LEAGUE,HOME_LEAGUE_ID,HOME_STRENGTH,HOME_LEAGUE_STRENGTH,...,HOME_TARGET,AWAY_TARGET,HOME_FOULS,AWAY_FOULS,HOME_CORNERS,AWAY_CORNERS,HOME_YELLOW,AWAY_YELLOW,HOME_RED,AWAY_RED
0,2021-01-01,Everton,West Ham,A,0.0,1.0,Premier League,E0,86.3,85.9,...,2.0,5.0,9.0,7.0,5.0,4.0,2.0,0.0,0.0,0.0
1,2021-01-01,Man United,Aston Villa,H,2.0,1.0,Premier League,E0,88.6,85.9,...,9.0,5.0,22.0,10.0,5.0,10.0,4.0,0.0,0.0,0.0
13,2021-01-02,Hertha,Schalke 04,H,3.0,0.0,Bundesliga,D1,77.5,84.0,...,10.0,4.0,11.0,18.0,9.0,5.0,2.0,2.0,0.0,0.0
19,2021-01-02,Sport Recife,Flamengo RJ,A,0.0,3.0,Serie A,BRA,76.2,80.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2021-01-02,Real Madrid,Celta,H,2.0,0.0,LaLiga,SP1,93.3,83.5,...,2.0,1.0,14.0,14.0,1.0,5.0,4.0,2.0,0.0,0.0


## Feature Engineering

We need to find a way to train the model to predict the results of the match completely blind. When actually predicting matches, the only data available will be the team/league strength and the stats from previous games. 

I will need to find a way to create features that involve the stats of the three previous matches for both the home and the away team in order to predict the result of the game.

In [195]:
home_df = match_data[[
    "DATE", "HOME_TEAM", "RESULT", "HOME_TEAM_SCORE", "AWAY_TEAM_SCORE",
    "HOME_STRENGTH", "HOME_LEAGUE_STRENGTH",
    "HOME_TARGET", "HOME_FOULS", "HOME_CORNERS", "HOME_YELLOW", "HOME_RED"
]].copy()
home_df.rename(columns=lambda x: x.replace("HOME_", ""), inplace=True)
home_df["TEAM"] = home_df["TEAM"]
home_df["VENUE"] = "HOME"

away_df = match_data[[
    "DATE", "AWAY_TEAM", "RESULT", "AWAY_TEAM_SCORE", "HOME_TEAM_SCORE",
    "AWAY_STRENGTH", "AWAY_LEAGUE_STRENGTH",
    "AWAY_TARGET", "AWAY_FOULS", "AWAY_CORNERS", "AWAY_YELLOW", "AWAY_RED"
]].copy()
away_df.rename(columns=lambda x: x.replace("AWAY_", ""), inplace=True)
away_df["TEAM"] = away_df["TEAM"]
away_df["VENUE"] = "AWAY"

# Standardize column names
away_df.columns = home_df.columns = [
    "DATE", "TEAM", "RESULT", "GOALS_FOR", "GOALS_AGAINST",
    "STRENGTH", "LEAGUE_STRENGTH",
    "TARGET", "FOULS", "CORNERS", "YELLOW", "RED", "VENUE"
]

matches_long = pd.concat([home_df, away_df]).sort_values(by="DATE")

home_df.head()
away_df.head()


,DATE,TEAM,RESULT,GOALS_FOR,GOALS_AGAINST,STRENGTH,LEAGUE_STRENGTH,TARGET,FOULS,CORNERS,YELLOW,RED,VENUE
0,2021-01-01,West Ham,A,1.0,0.0,85.5,85.9,5.0,7.0,4.0,0.0,0.0,AWAY
1,2021-01-01,Aston Villa,H,1.0,2.0,91.4,85.9,5.0,10.0,10.0,0.0,0.0,AWAY
13,2021-01-02,Schalke 04,H,0.0,3.0,75.7,84.0,4.0,18.0,5.0,2.0,0.0,AWAY
19,2021-01-02,Flamengo RJ,A,3.0,0.0,82.5,80.1,NaN,NaN,NaN,NaN,NaN,AWAY
18,2021-01-02,Celta,H,0.0,2.0,84.3,83.5,1.0,14.0,5.0,2.0,0.0,AWAY


In [196]:
# First, identify the numeric columns only
numeric_cols = [
    "GOALS_FOR", "GOALS_AGAINST",
    "STRENGTH", "LEAGUE_STRENGTH",
    "TARGET", "FOULS", "CORNERS", "YELLOW", "RED"
]

# Sort the data by team and date
matches_long = matches_long.sort_values(by=["TEAM", "DATE"])

# Group and compute rolling mean of past 3 games, excluding current match
rolling_features = (
    matches_long.groupby("TEAM")[numeric_cols]
    .apply(lambda group: group.shift(1).rolling(3).mean())
    .reset_index(level=0, drop=True)
)

# Rename columns to indicate they are historical aggregates
rolling_features.columns = [f"{col}_LAST3" for col in rolling_features.columns]

# Join back to matches_long
matches_long = pd.concat([matches_long, rolling_features], axis=1)

matches_long = matches_long.dropna(subset=[f"{col}_LAST3" for col in ["GOALS_FOR", "GOALS_AGAINST","STRENGTH", "LEAGUE_STRENGTH",]])

# Map points for form calculation
# 'H' = home win → 3 points for home team
# 'A' = away win → 3 points for away team

matches_long["POINTS"] = matches_long["RESULT"].map({"H": 3, "D": 1, "A": 0})

# Calculate rolling sum of last 3 points for each team (excluding current match)
matches_long["FORM_LAST3"] = (
    matches_long.groupby("TEAM")["POINTS"]
    .shift(1)  # exclude current match
    .rolling(window=3, min_periods=3)
    .sum()
    .reset_index(level=0, drop=True)
)

matches_long = matches_long.dropna(
    subset=[f"{col}_LAST3" for col in ["GOALS_FOR", "GOALS_AGAINST", "STRENGTH", "LEAGUE_STRENGTH"]] + ["FORM_LAST3"]
)

print(matches_long.shape)
print(matches_long.columns.tolist())
matches_long.head()


(31646, 24)
['DATE', 'TEAM', 'RESULT', 'GOALS_FOR', 'GOALS_AGAINST', 'STRENGTH', 'LEAGUE_STRENGTH', 'TARGET', 'FOULS', 'CORNERS', 'YELLOW', 'RED', 'VENUE', 'GOALS_FOR_LAST3', 'GOALS_AGAINST_LAST3', 'STRENGTH_LAST3', 'LEAGUE_STRENGTH_LAST3', 'TARGET_LAST3', 'FOULS_LAST3', 'CORNERS_LAST3', 'YELLOW_LAST3', 'RED_LAST3', 'POINTS', 'FORM_LAST3']


,DATE,TEAM,RESULT,GOALS_FOR,GOALS_AGAINST,STRENGTH,LEAGUE_STRENGTH,TARGET,FOULS,CORNERS,...,GOALS_AGAINST_LAST3,STRENGTH_LAST3,LEAGUE_STRENGTH_LAST3,TARGET_LAST3,FOULS_LAST3,CORNERS_LAST3,YELLOW_LAST3,RED_LAST3,POINTS,FORM_LAST3
210,2021-11-03,AFC Ajax,A,3.0,1.0,84.3,76.2,NaN,NaN,NaN,...,0.333333,84.3,76.2,NaN,NaN,NaN,NaN,NaN,0,3.0
230,2021-11-24,AFC Ajax,A,2.0,1.0,84.3,76.2,NaN,NaN,NaN,...,0.333333,84.3,76.2,NaN,NaN,NaN,NaN,NaN,0,4.0
240,2021-12-07,AFC Ajax,H,4.0,2.0,84.3,76.2,NaN,NaN,NaN,...,0.666667,84.3,76.2,NaN,NaN,NaN,NaN,NaN,3,4.0
265,2022-03-15,AFC Ajax,A,0.0,1.0,84.3,76.2,NaN,NaN,NaN,...,1.666667,84.3,76.2,NaN,NaN,NaN,NaN,NaN,0,9.0
16353,2024-08-31,AVS,H,1.0,2.0,75.7,78.6,2.0,9.0,2.0,...,1.666667,75.7,78.6,3.333333,11.666667,3.333333,2.666667,0.333333,3,4.0


Now we can remove all data pertaining to the current match, as we want the model to be able to predict the outcome based on the past three games.

In [197]:
# List of current match stats to drop
current_stats = [
    "GOALS_FOR", "GOALS_AGAINST",
    "TARGET", "FOULS", "CORNERS", "YELLOW", "RED"
]

# Drop them from the DataFrame
matches_long = matches_long.drop(columns=current_stats)

print(matches_long.columns.tolist())

['DATE', 'TEAM', 'RESULT', 'STRENGTH', 'LEAGUE_STRENGTH', 'VENUE', 'GOALS_FOR_LAST3', 'GOALS_AGAINST_LAST3', 'STRENGTH_LAST3', 'LEAGUE_STRENGTH_LAST3', 'TARGET_LAST3', 'FOULS_LAST3', 'CORNERS_LAST3', 'YELLOW_LAST3', 'RED_LAST3', 'POINTS', 'FORM_LAST3']


We should now join back to the original dataset to compare the team's and opponent's league and team strength.

In [201]:
# Separate home and away features
home_features = matches_long[matches_long["VENUE"] == "HOME"].copy()
away_features = matches_long[matches_long["VENUE"] == "AWAY"].copy()

# Drop unnecessary columns if already matched
home_features = home_features.drop(columns=["VENUE", "RESULT"])
away_features = away_features.drop(columns=["VENUE", "RESULT"])

# Rename columns
home_features = home_features.add_prefix("HOME_")
away_features = away_features.add_prefix("AWAY_")

# Join on DATE and TEAM match
combined_df = pd.merge(
    home_features,
    away_features,
    left_on="HOME_DATE",
    right_on="AWAY_DATE",
    suffixes=("", "_AWAY")
)

# Keep only one date column
combined_df["DATE"] = combined_df["HOME_DATE"]
combined_df.drop(columns=["HOME_DATE", "AWAY_DATE","HOME_POINTS","AWAY_POINTS"], inplace=True)

# Optionally, add the match result back from original dataset
result_df = match_data[["DATE", "HOME_TEAM", "AWAY_TEAM", "RESULT"]]
final_df = pd.merge(combined_df, result_df, on=["DATE", "HOME_TEAM", "AWAY_TEAM"])

#save the data to use for training
final_df.to_csv("../data/training_data.csv",index=False)

print(final_df.columns.tolist())
final_df.head()

['HOME_TEAM', 'HOME_STRENGTH', 'HOME_LEAGUE_STRENGTH', 'HOME_GOALS_FOR_LAST3', 'HOME_GOALS_AGAINST_LAST3', 'HOME_STRENGTH_LAST3', 'HOME_LEAGUE_STRENGTH_LAST3', 'HOME_TARGET_LAST3', 'HOME_FOULS_LAST3', 'HOME_CORNERS_LAST3', 'HOME_YELLOW_LAST3', 'HOME_RED_LAST3', 'HOME_FORM_LAST3', 'AWAY_TEAM', 'AWAY_STRENGTH', 'AWAY_LEAGUE_STRENGTH', 'AWAY_GOALS_FOR_LAST3', 'AWAY_GOALS_AGAINST_LAST3', 'AWAY_STRENGTH_LAST3', 'AWAY_LEAGUE_STRENGTH_LAST3', 'AWAY_TARGET_LAST3', 'AWAY_FOULS_LAST3', 'AWAY_CORNERS_LAST3', 'AWAY_YELLOW_LAST3', 'AWAY_RED_LAST3', 'AWAY_FORM_LAST3', 'DATE', 'RESULT']


,HOME_TEAM,HOME_STRENGTH,HOME_LEAGUE_STRENGTH,HOME_GOALS_FOR_LAST3,HOME_GOALS_AGAINST_LAST3,HOME_STRENGTH_LAST3,HOME_LEAGUE_STRENGTH_LAST3,HOME_TARGET_LAST3,HOME_FOULS_LAST3,HOME_CORNERS_LAST3,...,AWAY_STRENGTH_LAST3,AWAY_LEAGUE_STRENGTH_LAST3,AWAY_TARGET_LAST3,AWAY_FOULS_LAST3,AWAY_CORNERS_LAST3,AWAY_YELLOW_LAST3,AWAY_RED_LAST3,AWAY_FORM_LAST3,DATE,RESULT
0,AFC Ajax,84.3,76.2,3.000000,0.666667,84.3,76.2,NaN,NaN,NaN,...,88.8,78.6,NaN,NaN,NaN,NaN,NaN,4.0,2021-12-07,H
1,AFC Ajax,84.3,76.2,2.666667,1.666667,84.3,76.2,NaN,NaN,NaN,...,89.0,78.6,6.333333,11.333333,7.333333,2.000000,0.333333,9.0,2022-03-15,A
2,AVS,75.7,78.6,1.333333,2.000000,75.7,78.6,3.000000,12.000000,2.666667,...,78.3,78.6,2.000000,13.000000,6.000000,1.000000,0.333333,3.0,2024-09-14,H
3,AVS,75.7,78.6,0.333333,1.333333,75.7,78.6,1.000000,12.666667,4.666667,...,84.2,78.6,4.666667,16.333333,5.000000,2.666667,0.000000,2.0,2024-10-28,A
4,AVS,75.7,78.6,0.333333,2.000000,75.7,78.6,1.333333,11.333333,5.000000,...,80.4,78.6,3.000000,11.000000,6.333333,1.666667,0.000000,7.0,2024-11-03,A
